In [1]:
input_filename = "podaci/novi_podaci/post_naslovi.input"
dataset_filename = input_filename + '.prepared'
pretrained_model_name = 'gpt2'

In [13]:
import re
import alati

input_file = open(input_filename, "r")
output_file = open(dataset_filename, "w")

skip_filter = re.compile(r'(^|[^\w])(sunčano|kiša|oblačno|toplo|stepena|stepen|kišom|hladno|hladnije|stepeni|obilne padavine|toplije|sneg|ubije|ubil|poginu|silova|prebil|umrl|umro|preminu|mrtvi|dečak|devojčic|dete)', flags=re.I | re.M)

for line in input_file:
    line = alati.konvertuj_cirilicu(line)

    if skip_filter.search(line):
        continue

    line = line.replace('ž', 'ž').replace('š', 'š').replace('&quot;', '"').replace('&amp;', '&')
    line = re.sub(r'^(["”])*', '', line)
    line = re.sub(r'(["”])*$', '', line)
    line = line.replace('– ', ' ') \
        .replace('- ', ' ') \
        .replace('​', ' ')
        # .replace(': ', ' ') \

    line = line.replace('(KURIR TV)', '(VIDEO)').replace('KURIR TV', '')
    re.sub('za (kurir|blic)', 'za Fleš', line, flags=re.I)
    re.sub('uz (kurir|blic)', 'uz Fleš', line, flags=re.I)
    re.sub(' (kurir|blic) vam ', ' Fleš vam ', line, flags=re.I)
    re.sub('KURIR HOROSKOP', 'HOROSKOP', line, flags=re.I)

    output_file.write("<s>" + line.strip() + " </s>" + '\n')

input_file.close()
output_file.close()

In [3]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers import normalizers

tokenizer = ByteLevelBPETokenizer(lowercase=False)
tokenizer.normalizer = normalizers.Sequence([
    # normalizers.Strip(),
    normalizers.NFKC(),
])
tokenizer.train(
    files=[dataset_filename],
    min_frequency=20,
    vocab_size=50000,
    special_tokens=[
        "<s>",
        "<pad>",
        "</s>",
        # "<unk>",
        # "<mask>",
        # "<|endoftext|>",
    ],
)
tokenizer.save('naslovi-tokenizer.json')

In [14]:
from tokenizers import normalizers, pre_tokenizers, models, trainers, Tokenizer

model = models.WordLevel({}, '<unk>')
trainer = trainers.WordLevelTrainer(
    show_progress=True,
    min_frequency=20,
    vocab_size=50000,
    special_tokens=[
        "<s>",
        "<pad>",
        "</s>",
        "<unk>",
        # "<mask>",
        # "<|endoftext|>",
    ],
)

tokenizer = Tokenizer(model)
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
tokenizer.normalizer = normalizers.Sequence([
    # normalizers.Strip(),
    normalizers.NFKC(),
])
tokenizer.train(
    files=[dataset_filename],
    trainer=trainer
)
tokenizer.save('naslovi-simple-tokenizer.json')